In [7]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from sklearn.model_selection import train_test_split

# 1. Prepare Dataset and Tokenizer

def tokenize(text):
    return text.lower().split()  # very basic tokenizer

class TextDataset(Dataset):
    def __init__(self, texts, labels, vocab, max_len=100):
        self.texts = texts
        self.labels = labels
        self.vocab = vocab
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def encode(self, text):
        tokens = tokenize(text)
        idxs = [self.vocab.get(t, self.vocab['<unk>']) for t in tokens]
        # pad or truncate
        if len(idxs) < self.max_len:
            idxs = idxs + [self.vocab['<pad>']] * (self.max_len - len(idxs))
        else:
            idxs = idxs[:self.max_len]
        return torch.tensor(idxs)

    def __getitem__(self, idx):
        x = self.encode(self.texts[idx])
        y = torch.tensor(self.labels[idx])
        return x, y

# 2. Build Vocabulary

def build_vocab(texts, min_freq=1):
    counter = Counter()
    for text in texts:
        tokens = tokenize(text)
        counter.update(tokens)
    vocab = {'<pad>':0, '<unk>':1}
    for word, freq in counter.items():
        if freq >= min_freq:
            vocab[word] = len(vocab)
    return vocab

# 3. Model Definition

class SimpleTextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.fc1 = nn.Linear(embed_dim, 16)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(16, num_classes)

    def forward(self, x):
        # x shape: (batch_size, seq_len)
        embedded = self.embedding(x)  # (batch_size, seq_len, embed_dim)
        pooled = embedded.mean(dim=1)  # average pooling over seq_len
        out = self.relu(self.fc1(pooled))
        out = self.fc2(out)
        return out

# 4. Load Data and Train

# Replace with your own CSV file path
df = pd.read_csv('/Users/nguyennguyen/Desktop/github_repos/personal/rag_medical/src/data/emotion_data/training.csv')
texts = df['text'].astype(str).tolist()
labels = df['label'].tolist()

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Build vocab from training data
vocab = build_vocab(X_train)
print(f"Vocab size: {len(vocab)}")

# Hyperparameters
max_len = 100
embed_dim = 50
num_classes = len(set(labels))
batch_size = 32
epochs = 30
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Datasets and loaders
train_dataset = TextDataset(X_train, y_train, vocab, max_len)
test_dataset = TextDataset(X_test, y_test, vocab, max_len)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Model
model = SimpleTextClassifier(len(vocab), embed_dim, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * batch_x.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(batch_y).sum().item()
        total += batch_x.size(0)

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/total:.4f}, Accuracy: {correct/total:.4f}")

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch_x, batch_y in test_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        outputs = model(batch_x)
        _, predicted = outputs.max(1)
        correct += predicted.eq(batch_y).sum().item()
        total += batch_x.size(0)

print(f"Test Accuracy: {correct/total:.4f}")


Vocab size: 13529
Epoch 1/30, Loss: 1.6321, Accuracy: 0.3345
Epoch 2/30, Loss: 1.5572, Accuracy: 0.3405
Epoch 3/30, Loss: 1.4886, Accuracy: 0.4099
Epoch 4/30, Loss: 1.2877, Accuracy: 0.5259
Epoch 5/30, Loss: 1.0087, Accuracy: 0.6525
Epoch 6/30, Loss: 0.7835, Accuracy: 0.7362
Epoch 7/30, Loss: 0.6349, Accuracy: 0.7969
Epoch 8/30, Loss: 0.5306, Accuracy: 0.8359
Epoch 9/30, Loss: 0.4499, Accuracy: 0.8663
Epoch 10/30, Loss: 0.3816, Accuracy: 0.8868
Epoch 11/30, Loss: 0.3254, Accuracy: 0.9028
Epoch 12/30, Loss: 0.2786, Accuracy: 0.9180
Epoch 13/30, Loss: 0.2382, Accuracy: 0.9347
Epoch 14/30, Loss: 0.2028, Accuracy: 0.9458
Epoch 15/30, Loss: 0.1725, Accuracy: 0.9543
Epoch 16/30, Loss: 0.1474, Accuracy: 0.9615
Epoch 17/30, Loss: 0.1263, Accuracy: 0.9669
Epoch 18/30, Loss: 0.1090, Accuracy: 0.9723
Epoch 19/30, Loss: 0.0951, Accuracy: 0.9756
Epoch 20/30, Loss: 0.0833, Accuracy: 0.9793
Epoch 21/30, Loss: 0.0735, Accuracy: 0.9817
Epoch 22/30, Loss: 0.0655, Accuracy: 0.9838
Epoch 23/30, Loss: 0.05

In [8]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from sklearn.model_selection import train_test_split

# 1. Tokenizer and Dataset
def tokenize(text):
    return text.lower().split()

class TextDataset(Dataset):
    def __init__(self, texts, labels, vocab, max_len=100):
        self.texts = texts
        self.labels = labels
        self.vocab = vocab
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def encode(self, text):
        tokens = tokenize(text)
        idxs = [self.vocab.get(t, self.vocab['<unk>']) for t in tokens]
        if len(idxs) < self.max_len:
            idxs += [self.vocab['<pad>']] * (self.max_len - len(idxs))
        else:
            idxs = idxs[:self.max_len]
        return torch.tensor(idxs)

    def __getitem__(self, idx):
        x = self.encode(self.texts[idx])
        y = torch.tensor(self.labels[idx])
        return x, y

# 2. Vocabulary Builder
def build_vocab(texts, min_freq=1):
    counter = Counter()
    for text in texts:
        tokens = tokenize(text)
        counter.update(tokens)
    vocab = {'<pad>': 0, '<unk>': 1}
    for word, freq in counter.items():
        if freq >= min_freq:
            vocab[word] = len(vocab)
    return vocab

# 3. LSTM Model
class LSTMTextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        embedded = self.embedding(x)  # (batch_size, seq_len, embed_dim)
        _, (hidden, _) = self.lstm(embedded)  # hidden: (1, batch_size, hidden_dim)
        out = self.fc(hidden.squeeze(0))  # (batch_size, num_classes)
        return out

# 4. Load and Prepare Data
df = pd.read_csv('/Users/nguyennguyen/Desktop/github_repos/personal/rag_medical/src/data/emotion_data/training.csv')
texts = df['text'].astype(str).tolist()
labels = df['label'].tolist()

X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)
vocab = build_vocab(X_train)

print(f"Vocab size: {len(vocab)}")

# 5. Hyperparameters
max_len = 100
embed_dim = 50
hidden_dim = 64
num_classes = len(set(labels))
batch_size = 32
epochs = 30
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 6. Datasets and Loaders
train_dataset = TextDataset(X_train, y_train, vocab, max_len)
test_dataset = TextDataset(X_test, y_test, vocab, max_len)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# 7. Model, Loss, Optimizer
model = LSTMTextClassifier(len(vocab), embed_dim, hidden_dim, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 8. Training Loop
for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * batch_x.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(batch_y).sum().item()
        total += batch_x.size(0)

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/total:.4f}, Accuracy: {correct/total:.4f}")

# 9. Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch_x, batch_y in test_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        outputs = model(batch_x)
        _, predicted = outputs.max(1)
        correct += predicted.eq(batch_y).sum().item()
        total += batch_x.size(0)

print(f"Test Accuracy: {correct/total:.4f}")


Vocab size: 13529
Epoch 1/30, Loss: 1.5865, Accuracy: 0.3329
Epoch 2/30, Loss: 1.5732, Accuracy: 0.3380
Epoch 3/30, Loss: 1.5728, Accuracy: 0.3384
Epoch 4/30, Loss: 1.5725, Accuracy: 0.3391
Epoch 5/30, Loss: 1.5719, Accuracy: 0.3391
Epoch 6/30, Loss: 1.5728, Accuracy: 0.3364
Epoch 7/30, Loss: 1.5723, Accuracy: 0.3380
Epoch 8/30, Loss: 1.5722, Accuracy: 0.3391
Epoch 9/30, Loss: 1.5721, Accuracy: 0.3391
Epoch 10/30, Loss: 1.5721, Accuracy: 0.3391
Epoch 11/30, Loss: 1.5718, Accuracy: 0.3391
Epoch 12/30, Loss: 1.5722, Accuracy: 0.3391
Epoch 13/30, Loss: 1.5718, Accuracy: 0.3391
Epoch 14/30, Loss: 1.5721, Accuracy: 0.3391
Epoch 15/30, Loss: 1.5720, Accuracy: 0.3391
Epoch 16/30, Loss: 1.5719, Accuracy: 0.3391
Epoch 17/30, Loss: 1.5715, Accuracy: 0.3391
Epoch 18/30, Loss: 1.5717, Accuracy: 0.3391
Epoch 19/30, Loss: 1.5720, Accuracy: 0.3391
Epoch 20/30, Loss: 1.5715, Accuracy: 0.3391
Epoch 21/30, Loss: 1.5721, Accuracy: 0.3391
Epoch 22/30, Loss: 1.5720, Accuracy: 0.3391
Epoch 23/30, Loss: 1.57

In [10]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from sklearn.model_selection import train_test_split

# 1. Tokenizer and Dataset
def tokenize(text):
    return text.lower().split()

class TextDataset(Dataset):
    def __init__(self, texts, labels, vocab, max_len=100):
        self.texts = texts
        self.labels = labels
        self.vocab = vocab
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def encode(self, text):
        tokens = tokenize(text)
        idxs = [self.vocab.get(t, self.vocab['<unk>']) for t in tokens]
        if len(idxs) < self.max_len:
            idxs += [self.vocab['<pad>']] * (self.max_len - len(idxs))
        else:
            idxs = idxs[:self.max_len]
        return torch.tensor(idxs)

    def __getitem__(self, idx):
        x = self.encode(self.texts[idx])
        y = torch.tensor(self.labels[idx])
        return x, y

# 2. Vocabulary Builder
def build_vocab(texts, min_freq=1):
    counter = Counter()
    for text in texts:
        tokens = tokenize(text)
        counter.update(tokens)
    vocab = {'<pad>': 0, '<unk>': 1}
    for word, freq in counter.items():
        if freq >= min_freq:
            vocab[word] = len(vocab)
    return vocab

# 3. RNN Model
class RNNTextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.rnn = nn.RNN(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        embedded = self.embedding(x)  # (batch_size, seq_len, embed_dim)
        output, hidden = self.rnn(embedded)  # hidden: (1, batch_size, hidden_dim)
        out = self.fc(hidden.squeeze(0))  # (batch_size, num_classes)
        return out

# 4. Load and Prepare Data
df = pd.read_csv('/Users/nguyennguyen/Desktop/github_repos/personal/rag_medical/src/data/emotion_data/training.csv')
texts = df['text'].astype(str).tolist()
labels = df['label'].tolist()

X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)
vocab = build_vocab(X_train)

print(f"Vocab size: {len(vocab)}")

# 5. Hyperparameters
max_len = 100
embed_dim = 50
hidden_dim = 64
num_classes = len(set(labels))
batch_size = 32
epochs = 30
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 6. Datasets and Loaders
train_dataset = TextDataset(X_train, y_train, vocab, max_len)
test_dataset = TextDataset(X_test, y_test, vocab, max_len)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# 7. Model, Loss, Optimizer
model = RNNTextClassifier(len(vocab), embed_dim, hidden_dim, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 8. Training Loop
for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * batch_x.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(batch_y).sum().item()
        total += batch_x.size(0)

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/total:.4f}, Accuracy: {correct/total:.4f}")

# 9. Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch_x, batch_y in test_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        outputs = model(batch_x)
        _, predicted = outputs.max(1)
        correct += predicted.eq(batch_y).sum().item()
        total += batch_x.size(0)

print(f"Test Accuracy: {correct/total:.4f}")


Vocab size: 13529
Epoch 1/30, Loss: 1.5831, Accuracy: 0.3282
Epoch 2/30, Loss: 1.5749, Accuracy: 0.3362
Epoch 3/30, Loss: 1.5739, Accuracy: 0.3341
Epoch 4/30, Loss: 1.5738, Accuracy: 0.3380
Epoch 5/30, Loss: 1.5735, Accuracy: 0.3373
Epoch 6/30, Loss: 1.5728, Accuracy: 0.3391
Epoch 7/30, Loss: 1.5731, Accuracy: 0.3374
Epoch 8/30, Loss: 1.5731, Accuracy: 0.3376
Epoch 9/30, Loss: 1.5726, Accuracy: 0.3370
Epoch 10/30, Loss: 1.5724, Accuracy: 0.3374
Epoch 11/30, Loss: 1.5727, Accuracy: 0.3391
Epoch 12/30, Loss: 1.5731, Accuracy: 0.3370
Epoch 13/30, Loss: 1.5724, Accuracy: 0.3391
Epoch 14/30, Loss: 1.5722, Accuracy: 0.3379
Epoch 15/30, Loss: 1.5727, Accuracy: 0.3391
Epoch 16/30, Loss: 1.5725, Accuracy: 0.3391
Epoch 17/30, Loss: 1.5722, Accuracy: 0.3391
Epoch 18/30, Loss: 1.5829, Accuracy: 0.3264
Epoch 19/30, Loss: 1.5780, Accuracy: 0.3298
Epoch 20/30, Loss: 1.5775, Accuracy: 0.3267
Epoch 21/30, Loss: 1.5763, Accuracy: 0.3323
Epoch 22/30, Loss: 1.5766, Accuracy: 0.3317
Epoch 23/30, Loss: 1.57

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from sklearn.model_selection import train_test_split

# 1. Tokenizer and Dataset
def tokenize(text):
    return text.lower().split()

class TextDataset(Dataset):
    def __init__(self, texts, labels, vocab, max_len=100):
        self.texts = texts
        self.labels = labels
        self.vocab = vocab
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def encode(self, text):
        tokens = tokenize(text)
        idxs = [self.vocab.get(t, self.vocab['<unk>']) for t in tokens]
        if len(idxs) < self.max_len:
            idxs += [self.vocab['<pad>']] * (self.max_len - len(idxs))
        else:
            idxs = idxs[:self.max_len]
        return torch.tensor(idxs)

    def __getitem__(self, idx):
        x = self.encode(self.texts[idx])
        y = torch.tensor(self.labels[idx])
        return x, y

# 2. Vocabulary Builder
def build_vocab(texts, min_freq=1):
    counter = Counter()
    for text in texts:
        tokens = tokenize(text)
        counter.update(tokens)
    vocab = {'<pad>': 0, '<unk>': 1}
    for word, freq in counter.items():
        if freq >= min_freq:
            vocab[word] = len(vocab)
    return vocab

# 3. CNN Model
class CNNTextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes, kernel_sizes=[3, 4, 5], num_filters=100):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.convs = nn.ModuleList([
            nn.Conv2d(1, num_filters, (k, embed_dim)) for k in kernel_sizes
        ])
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(num_filters * len(kernel_sizes), num_classes)

    def forward(self, x):
        x = self.embedding(x)  # (batch_size, seq_len, embed_dim)
        x = x.unsqueeze(1)  # (batch_size, 1, seq_len, embed_dim)
        conv_x = [torch.relu(conv(x)).squeeze(3) for conv in self.convs]
        pooled_x = [torch.max(c, dim=2)[0] for c in conv_x]
        cat_x = torch.cat(pooled_x, dim=1)
        out = self.dropout(cat_x)
        return self.fc(out)

# 4. Load Data
df = pd.read_csv('/Users/nguyennguyen/Desktop/github_repos/personal/rag_medical/src/data/emotion_data/training.csv')
texts = df['text'].astype(str).tolist()
labels = df['label'].tolist()

X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)
vocab = build_vocab(X_train)

print(f"Vocab size: {len(vocab)}")

# 5. Hyperparameters
max_len = 100
embed_dim = 50
num_classes = len(set(labels))
batch_size = 32
epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 6. Datasets and Loaders
train_dataset = TextDataset(X_train, y_train, vocab, max_len)
test_dataset = TextDataset(X_test, y_test, vocab, max_len)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# 7. Model, Loss, Optimizer
model = CNNTextClassifier(len(vocab), embed_dim, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 8. Training Loop
for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * batch_x.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(batch_y).sum().item()
        total += batch_x.size(0)

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/total:.4f}, Accuracy: {correct/total:.4f}")

# 9. Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch_x, batch_y in test_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        outputs = model(batch_x)
        _, predicted = outputs.max(1)
        correct += predicted.eq(batch_y).sum().item()
        total += batch_x.size(0)

print(f"Test Accuracy: {correct/total:.4f}")


Vocab size: 13529
Epoch 1/10, Loss: 1.6296, Accuracy: 0.3421
Epoch 2/10, Loss: 1.4195, Accuracy: 0.4501
Epoch 3/10, Loss: 1.0753, Accuracy: 0.6048
Epoch 4/10, Loss: 0.6735, Accuracy: 0.7624
Epoch 5/10, Loss: 0.4138, Accuracy: 0.8560
Epoch 6/10, Loss: 0.2944, Accuracy: 0.8977
Epoch 7/10, Loss: 0.2178, Accuracy: 0.9234
Epoch 8/10, Loss: 0.1720, Accuracy: 0.9405
Epoch 9/10, Loss: 0.1428, Accuracy: 0.9493
Epoch 10/10, Loss: 0.1202, Accuracy: 0.9572
Test Accuracy: 0.8919


In [2]:
df_validation = pd.read_csv('/Users/nguyennguyen/Desktop/github_repos/personal/rag_medical/src/data/emotion_data/validation.csv')

texts_validation = df_validation['text'].astype(str).tolist()
labels_validation = df_validation['label'].tolist()
validation_dataset = TextDataset(texts_validation, labels_validation, vocab, max_len)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size)
# 10. Validation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch_x, batch_y in validation_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        outputs = model(batch_x)
        _, predicted = outputs.max(1)
        correct += predicted.eq(batch_y).sum().item()
        total += batch_x.size(0)
print(f"Validation Accuracy: {correct/total:.4f}")

Validation Accuracy: 0.8920
